In [2]:
import pandas as pd
import datetime

# Load data
data = pd.read_csv('good_driver.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Define the time segments
time_segments = {
    'rush_hours': [(8,10), (17,20)],
    'daytime': [(10,17)],
    'night': [(0,8), (19,24)]
}

score_adjustments = {
    'rush_hours': {'sudden_acceleration': -1, 'sudden_deceleration': -1, 'short_honk': -0.5, 'long_honk': -1},
    'daytime': {'sudden_acceleration': -2, 'sudden_deceleration': -2, 'short_honk': -1, 'long_honk': -2},
    'night': {'sudden_acceleration': -3, 'sudden_deceleration': -3, 'short_honk': -2, 'long_honk': -4}
}

def classify_score(score_value):
    if score_value > 80:
        return "Excellent"
    elif score_value > 60:
        return "Good"
    elif score_value > 40:
        return "Fair"
    else:
        return "Poor"


def calculate_daily_score(data_for_day):
    score = 100
    segmented_events = {'rush_hours': [], 'daytime': [], 'night': []}

    # Segment events
    for _, event in data_for_day.iterrows():
        time_matched = False
        for segment, time_ranges in time_segments.items():
            for time_range in time_ranges:
                if time_range[0] <= event['timestamp'].hour < time_range[1]:
                    segmented_events[segment].append(event['event_type'])
                    time_matched = True
                    break  # break out of the inner loop as soon as a time match is found
            if time_matched:
                break  # break out of the outer loop if a time match is found to avoid double-counting

    # Score adjustments
    for segment, events_list in segmented_events.items():
        for event in events_list:
            score += score_adjustments[segment].get(event, 0)

    # Evaluate excessive honking
    honk_times = data_for_day[data_for_day['event_type'].str.contains('honk')]['timestamp'].tolist()
    for i, honk_time in enumerate(honk_times[:-1]):
        count_in_10_minutes = sum(1 for next_honk_time in honk_times[i+1:] if (next_honk_time - honk_time) <= datetime.timedelta(minutes=10))
        if count_in_10_minutes > 5:
            score -= 5  # reduce once for every 10-minute interval with excessive honking

    # Bonus for every 5 minutes of penalty-free driving
    timestamps = data_for_day['timestamp'].tolist()
    penalty_events = ['sudden_acceleration', 'sudden_deceleration', 'short_honk', 'long_honk']

    # Initialize the start time
    start_time = timestamps[0]

    for time in timestamps[1:]:
        # Check if there's any penalty event between start_time and time
        penalty_in_interval = any(data_for_day[(data_for_day['timestamp'] >= start_time) & 
                                               (data_for_day['timestamp'] < time)]['event_type'].isin(penalty_events))
        # If no penalty and duration is 5 minutes, award bonus
        if (time - start_time) >= datetime.timedelta(minutes=5) and not penalty_in_interval:
            score += 1
            score = min(100, score)  # Ensure score doesn't exceed 100
            start_time = time  # Reset start_time for next interval
        elif penalty_in_interval:
            start_time = time  # Reset start_time due to penalty

    # Classification
    classification = classify_score(score)

    return score, classification



def calculate_weekly_score(data):
    daily_scores = []

    # Get the latest date from the dataset
    latest_date = data['timestamp'].dt.date.max()
    
    # Calculate the starting date for the 7-day window
    start_date = latest_date - datetime.timedelta(days=6)

    # Loop through the last 7 days
    for i in range(7):
        day = start_date + datetime.timedelta(days=i)
        daily_events = data[data['timestamp'].dt.date == day]
        score, _ = calculate_daily_score(daily_events)
        daily_scores.append(score)
        print(f"Daily Score for {day}: {score}")
        # Classification
        classification = classify_score(score)
        print(f"Classified Driver for {day}: {classification}")

    weekly_score = sum(daily_scores) / len(daily_scores)

    return weekly_score





# Test
#daily_score, daily_classification = calculate_daily_score(data)
#print(f"Daily Score: {daily_score}, Classification: {daily_classification}")

weekly_score = calculate_weekly_score(data)
print(f"Weekly Average Score: {weekly_score}")

# Calculate the event count
event_count = data['event_type'].value_counts()

print(event_count)

Daily Score for 2023-08-31: 80.0
Classified Driver for 2023-08-31: Good
Daily Score for 2023-09-01: 79.0
Classified Driver for 2023-09-01: Good
Daily Score for 2023-09-02: 77.5
Classified Driver for 2023-09-02: Good
Daily Score for 2023-09-03: 65.0
Classified Driver for 2023-09-03: Good
Daily Score for 2023-09-04: 93.0
Classified Driver for 2023-09-04: Excellent
Daily Score for 2023-09-05: 83.5
Classified Driver for 2023-09-05: Excellent
Daily Score for 2023-09-06: 96
Classified Driver for 2023-09-06: Excellent
Weekly Average Score: 82.0
short_honk             193
sudden_acceleration     55
sudden_deceleration     41
long_honk                5
Name: event_type, dtype: int64
